In [1]:
import csv
import nbimporter
import math
import sys
import os
sys.path.append('../')
from module import sun_position

Importing Jupyter notebook from ..\module\sun_position.ipynb


In [2]:
def get_directory_path():
    if os.path.isdir('./SCFConfig01'):
        return './SCFConfig01/'
    elif os.path.isdir('./module/SCFConfig01'):
        return './module/SCFConfig01/'        

# Functions

## read_input_poind_config

### Outline

Zone.csv の説明
- 1行目はヘッダ：地域区分, 都市, 緯度, 経度, 日射量ファイル名, 暖房開始日, 暖房終了日, 冷房開始日, 冷房終了日
- 2～9行目：1～8地域の「地域区分, 都市, 緯度, 経度, 日射量ファイル名, 暖房開始日, 暖房終了日, 冷房開始日, 冷房終了日」

暖冷房開始日終了日の書式：5桁もしくは6桁の数字
      
- 気象データの日時表記と同じ
- 後ろから2桁：時刻
- 後ろから3～4桁：日
- 後ろから5～6桁：月
- 気象データの「暖房=1,冷房2」の設定は上書き処理される → `Zone.csv` の設定が優先

地点データを \Zone.csv から読み込む  
\地域区分+日射量データ窓面入射角特性.xlsx "地域区分"シート
→ \SCFConfig01 下の地点データファイル \Zone.csv を作成 → 読み込み

### Function

In [3]:
def input_Point(tag):
    
    def convert_tag_to_city(tag):
        if type(tag) == int: # name に数字が指定された場合は省エネ基準の地域区分番号とみなして該当する都市に置き換える
            return {
                '1' : '北見',
                '2' : '岩見沢',
                '3' : '盛岡',
                '4' : '長野',
                '5' : '宇都宮',
                '6' : '岡山',
                '7' : '宮崎',
                '8' : '那覇'
            }[str(tag)]
        else: # その他の場合は都市名が入力されているものとし、変換は行わない
            return tag
    
    ZONE_FILE = get_directory_path() + 'Zone.csv'
    
    city_name = convert_tag_to_city(tag)

    with open(ZONE_FILE, 'r') as f:
        
        reader = csv.reader(f)
        header = next(reader) # Skip header.

        matched_rows = [ row for row in reader if row[1] == city_name ]  # Extract row matched.
        
        if len(matched_rows) == 0:  # In the case that there is now matched data.
            raise IndexError("地点データがありません")
        elif len(matched_rows) > 1:  # In the case that there are more than 1 matched data.
            raise IndexError("該当する地点データが複数見つかりました")
        else:
            matched_row = matched_rows[0]
            return [
                int(matched_row[0]), # region
                matched_row[1],      # city name
                float(matched_row[2]), # latitude
                float(matched_row[3]), # longitude
                matched_row[4],      # file name
                int(matched_row[5]), # start of heating period
                int(matched_row[6]), # end of heating period
                int(matched_row[7]), # start of cooling period
                int(matched_row[8])  # end of cooling period
            ]
            # ここで返るSRFileNameはファイル名のみ

### Example 1

省エネルギー基準における地域区分の番号で指定した場合

In [4]:
for i in range(1,9):
    print( input_Point( i ) )

[1, '北見', 43.82, 143.91, 'SRforSCF_01.csv', 110100, 43024, 50100, 103124]
[2, '岩見沢', 43.21, 141.78833, 'SRforSCF_02.csv', 110100, 43024, 50100, 103124]
[3, '盛岡', 39.695, 141.168333333333, 'SRforSCF_03.csv', 120100, 33124, 40100, 113024]
[4, '長野', 36.66, 138.195, 'SRforSCF_04.csv', 120100, 33124, 40100, 113024]
[5, '宇都宮', 36.5466666666667, 139.871666666667, 'SRforSCF_05.csv', 120100, 33124, 40100, 113024]
[6, '岡山', 34.6583333333333, 133.918333333333, 'SRforSCF_06.csv', 120100, 33124, 40100, 113024]
[7, '宮崎', 31.935, 131.416666666667, 'SRforSCF_07.csv', 120100, 33124, 40100, 113024]
[8, '那覇', 26.2033333333333, 127.688333333333, 'SRforSCF_08.csv', 10100, 33124, 40100, 123124]


### Example 2

都市名で指定した場合

In [5]:
print(input_Point( "宇都宮"))
print(input_Point( "那覇" ))

[5, '宇都宮', 36.5466666666667, 139.871666666667, 'SRforSCF_05.csv', 120100, 33124, 40100, 113024]
[8, '那覇', 26.2033333333333, 127.688333333333, 'SRforSCF_08.csv', 10100, 33124, 40100, 123124]


### Example 3

都市名で指定したが該当する名前がリストになかった場合

In [6]:
try:
    input_Point( "東京" ) #データなし
except IndexError as e:
    print(e)

地点データがありません


## input_SRData

### Outline

`\SRforSCF_\*\*.csv`
- \*\*は、1～8地域に対応して01～08の数字が入る(基本)。
- `\Zone.csv` に追加して対応することは可能。   
    
- 法線面直達日射量、水平面天空日射量、暖房期or冷房期の判別タグ(暖房期:1, 冷房期:2, 非空調期:0)
- 日射量の単位は kcal/(m<sup>2</sup>h)] → 効果係数算定には問題ないのでそのまま使用している
    
    
- 1行目はヘッダ：`\SRforSCF_\*\*.csv`(ファイル名), 法線面直達日射量, 水平面天空日射量, 暖房1_冷房2
- 2～8762行目：日時, 法線面直達日射量, 水平面天空日射量, 暖房1_冷房2
- 2行目が1月1日0時、8762行目が12月31日24時。1時間間隔。全8761データ
- 気象データファイル中の「暖房1冷房2」の設定は、`\Zone.csv` の設定で上書きされる → 現時点で意味なし
- 1列目の日時から「月」を算出

気象データを \SRforSCF_**.csv から読み込む  

\地域区分+日射量データ窓面入射角特性.xlsx "SRforSCF_**.csv"シート 

→ \SCFConfig01 下の地点データファイル \SRforSCF_**.csv を作成 → 読み込み

### Function

In [7]:
def input_SRData(file_name, h_period_start, h_period_end, c_period_start, c_period_end):

    def judge_heating_or_cooling_period(day_time):
        
        def is_cooling_period(day_time):
            
            # 冷房期間内の場合
            if c_period_start <= day_time <= c_period_end:
                return True
            
            # 冷房期間が年をまたぐ場合
            if c_period_start > c_period_end:
                if c_period_start <= day_time:
                    return True
                if day_time <= c_period_end:
                    return True
            
            return False
        
        def is_heating_period(day_time):

            # 暖房期間内の場合
            if h_period_start <= day_time <= h_period_end:
                return True
            
            # 暖房期間が年をまたぐ場合
            if h_period_start > h_period_end:
                if h_period_start <= day_time:
                    return True
                if day_time <= h_period_end:
                    return True

            return False
                
        HeatingPeriod, CoolingPeriod, NonACPeriod = 1, 2, 0

        if is_cooling_period(day_time):
            return CoolingPeriod 
        elif is_heating_period(day_time):
            return HeatingPeriod
        else:
            # 上記のいずれのケースにも当てはまらない場合 = 非暖冷房期間
            return NonACPeriod

    # n: 1/1 0:00 を 0 とするindex. 12/31 24:00 まであるため、0~8760 の 8761 個の値である。
    def make_new_row(n, row):
        return [
            n,
            int(row[0]),
            int(row[1]),
            int(row[2]),
            judge_heating_or_cooling_period(int(row[0]))
        ]
    
    # file_name = "SRforSCF_**.csv"
    
    PATH = get_directory_path()
    # PATH = './SCFConfig01/'
    
    file_path = PATH + file_name
    
    with open(file_path, 'r') as f:
        
        reader = csv.reader(f)
        header = next(reader) # Skip header.
    
        # \Zone.csv の設定で、暖房期,冷房期,非空調期を割り当て
        # 元ファイルの4列目はなかったことになる。

        return [make_new_row(n, row) for (n, row) in enumerate(reader)] # 8761 行をもつ

#### Example

In [8]:
for i in range(1,9):
    
    Zone, City, Latitude, Longitude, SRFileName, HStart, HEnd, CStart, CEnd = input_Point(i)
    
    print(SRFileName)
    
    print('Expected start of heating period: ' + str(HStart) )
    print('Expected end of heating period: ' + str(HEnd) )
    print('Expected start of cooling period: ' + str(CStart) )
    print('Expected end of cooling period: ' + str(CEnd) )
    
    SRHour = input_SRData(SRFileName, HStart, HEnd, CStart, CEnd)

    for i in range(1,len(SRHour)-1):
        if SRHour[i][4] != SRHour[i-1][4]:
            print(SRHour[i-1][1], SRHour[i-1][4], SRHour[i][1], SRHour[i][4])    

SRforSCF_01.csv
Expected start of heating period: 110100
Expected end of heating period: 43024
Expected start of cooling period: 50100
Expected end of cooling period: 103124
43024 1 50101 2
103124 2 110101 1
SRforSCF_02.csv
Expected start of heating period: 110100
Expected end of heating period: 43024
Expected start of cooling period: 50100
Expected end of cooling period: 103124
43024 1 50101 2
103124 2 110101 1
SRforSCF_03.csv
Expected start of heating period: 120100
Expected end of heating period: 33124
Expected start of cooling period: 40100
Expected end of cooling period: 113024
33124 1 40101 2
113024 2 120101 1
SRforSCF_04.csv
Expected start of heating period: 120100
Expected end of heating period: 33124
Expected start of cooling period: 40100
Expected end of cooling period: 113024
33124 1 40101 2
113024 2 120101 1
SRforSCF_05.csv
Expected start of heating period: 120100
Expected end of heating period: 33124
Expected start of cooling period: 40100
Expected end of cooling period: 1

## calc_Month

#### Function

In [9]:
#「月」の計算 """
def calc_Month(MMDDTT):
    return MMDDTT // 10000

## D.3 正時±30分で太陽が地平線上にある時間刻み数のカウント (仕様書6.2 式(3)及び図5中の$n_H$の計算)

- 算定ツール標準の時間分割数$n_{\Delta t}$は、$6$ ($10$分刻み)
  - $1$時間を$1$分割もしくは$2$以上の偶数で分割する  
  
- 正時$\pm 30$分間で太陽が地平線上にある(太陽高度$>0$)時間刻み数をカウントして、$n_H$を計算

In [10]:
def calc_Nh(sinh, prev_sinh, NDT):

    if NDT == 1:
        if sinh[0] > 0.0:
            return 1.0
        else:
            return 0
    elif NDT > 0 and NDT % 2 == 0:
        sinh0 = []
        sinh0.extend([prev_sinh[m] for m in range(int(NDT/2),int(NDT))])
        sinh0.extend([sinh[m] for m in range(int(NDT/2)+1)])
        return ( sum(x > 0 for x in sinh0)
                - (0.5 if sinh0[0] > 0 else 0)
                - (0.5 if sinh0[int(NDT)] > 0 else 0)
               )
    else:
        raise ValueError('1時間あたりの時間分割数は1もしくは2以上の偶数とする必要があります')

## D.4 $1/n_{\Delta t}$時間間隔での日射量 (仕様書6.2 式(3)の計算, 図5参照)

- 算定ツール標準の時間分割数$n_{\Delta t}$は、$6$ ($10$分刻み)
- 正時$\pm 30$分間で太陽が地平線上にある(太陽高度$>0$)時間刻み数をカウントして、$n_H$を計算
- 法線面直達日射量, 水平面天空日射量をそれぞれに適用し、時間刻みにおける日射量を算定する

In [11]:
def calc_Sdhm(MM, NDT, sinh, Sh, Shp, Nh, Nhp):
    # MM:1時間の内の(1/NDT)間隔の順番, 正時がMM=0, MM=0～NDT-1
    # NDT = 6                    # 1時間の分割数,ツールの標準は6分割
    # sinh:hsdtの正弦
    # Sh: 法線面直達日射量[kcal/(m2h)]
    # Shp: 法線面直達日射量[kcal/(m2h)] 1時間後
    # Nh: 太陽が地平線上にある時間（分割した後の⊿tの数）
    # Nhp: 太陽が地平線上にある時間（分割した後の⊿tの数） 1時間後
    
    if sinh > 0:
        return ( ( (Sh  / Nh  if MM <= NDT/2 and Nh  > 0 else 0)
                 + (Shp / Nhp if MM >= NDT/2 and Nhp > 0 else 0) )
               / (2 if MM == NDT/2 else 1) )
        #以下のコメントアウト部分を上式で置き換え
        #if MM < NDT/2 and Nh > 0:
        #    Sdhm = Sh / Nh
        #elif MM == NDT/2:
        #    if Nh > 0:
        #        Sdhm += Sh / Nh / 2
        #    if Nhp > 0:
        #        Sdhm += Shp / Nhp / 2            
        #elif NDT/2 < MM and Nhp > 0:
        #    Sdhm = Shp / Nhp
    else:
        return 0.0

## D.5 窓ガラスの入射角特性

IncidentAngleCharacteristics クラスは以下のプロパティをもつ。
- 直達日射に対する入射角特性最大値(入射角0) $\eta_{max}$
- 天空・反射日射に対する入射角特性(遮蔽なしの場合) $\eta_{isr}$
- $\eta_{j,d,t}(\theta_{j,d,t})$ 算定式の係数$k_n$$(n=0～7)$

$$\eta_{j,d,t}(\theta_{j,d,t}) = \sum_{n=0}^7 k_n \cos^n \theta_{j,d,t}$$
       
  - デフォルトとして、以下を設定
    - $ID=0$：日よけ効果係数内で入射角特性非考慮 → $\eta_{j,d,t}(\theta_{j,d,t}) = 1$
    - $ID=1$：解説書の入射角特性(「平成25年度省エネルギー基準に準拠した算定・判断の方法及び解説 I 非住宅建築物 第二版(連合印刷センター, 平成26年○月○日)」, pp.168-170, 式(2.1.25),(2.1.28),(2.1.32))
    
       $$\eta_{j,d,t}(\theta_{j,d,t}) = 2.3920 \cos \theta_{j,d,t} -3.8636 \cos^3 \theta_{j,d,t} + 3.7568 \cos^5 \theta_{j,d,t} - 1.3952 \cos^7 \theta_{j,d,t} $$

- 他の特性を入れる際には、$ID$を違えて、`\IncidentAngleCharacteristics.csv` に追加する。

In [12]:
class IncidentAngleCharacteristics():
    
    def __init__(self, eta_max, eta_isr, eta_kk):
        self.__eta_max = eta_max
        self.__eta_isr = eta_isr
        self.__eta_kk = eta_kk
    
    @property
    def eta_max(self):
        return self.__eta_max
    
    @property
    def eta_isr(self):
        return self.__eta_isr
    
    @property
    def eta_kk(self):
        return self.__eta_kk

In [13]:
def get_incident_angle_characteristics():
    return IncidentAngleCharacteristics(eta_max = 0.88,
                                        eta_isr = 0.808,
                                        eta_kk = [0.0, 2.392, 0.0, -3.8636, 0.0, 3.7568, 0.0, -1.3952]
                                       )

## D.6 直達日射に対する窓ガラスの入射角特性 (緑本非住宅第二版pp.169 式(2.1.28)準拠)

- 「平成25年度省エネルギー基準に準拠した算定・判断の方法及び解説 I 非住宅建築物 第二版(連合印刷センター, 平成26年○月○日)」pp.168-170参照
- 入射角$\theta_{j,d,t}[deg]$, 太陽高度$h_{S,d,t}[deg]$, 太陽方位角$A_{ZS,d,t}[deg]$, 外壁$j$の方位角$A_{ZW,i}[deg]$
- 入射角特性算定式の係数$k_n$$(n=0～7)$, 日付$d$時刻$t$における入射角特性$\eta_{j,d,t}$

$$\cos \theta_{j,d,t} = \cos h_{S,d,t} \cos (A_{ZS,d,t} - A_{ZW,i}) \qquad (2.1.26) $$
$$\eta_{j,d,t}(\theta_{j,d,t}) = \sum_{n=0}^7 k_n \cos^n \theta_{j,d,t} \qquad (2.1.28') $$

In [14]:
def calc_costheta(Azwjdt, cosh):

    return max(cosh * math.cos(math.radians(Azwjdt)),0.0)

def calc_etajdt(costheta, etakk):

    return sum([eta * costheta**i for (i,eta) in enumerate(etakk)])


# Classes

## SunData

In [15]:
class SunData():
    
    def __init__(self, index, day_time, direct, sky, hc_period, latitude, longitude, ndt):
        
        # index: 1/1 0:00 を 0 とするindex. 12/31 24:00 まであるため、0~8760 の 8761 個の値である。
        self.__index = index
        self.__nday, self.__nhour = sun_position.calc_NDayNHour(index)
        self.__day_time = day_time
        self.__direct = direct
        self.__sky = sky
        self.__hc_period = hc_period
        
        # 時刻(h)TT データの作成
        self.__tt = [sun_position.calc_TT(self.__nhour, ndt, mm) for mm in range(ndt)]

        # 太陽高度 hsdt (deg) の正弦
        self.__sinh = []
        # 太陽高度 hsdt (deg) の余弦
        self.__cosh = []
        # 太陽高度 hsdt
        self.__hsdt = []
        # 太陽方位角 Azsdt (deg)
        self.__azsdt = []
        
        for tt in self.__tt:
            year = 2017 # HASP 計算式では年を使用していないため、この値はなんでもよい。
            longitude_std = 135.0
            method = 'HASP'
            hs, A = sun_position.calc_solar_position(
                year,
                self.__nday,
                tt,
                math.radians(latitude),
                math.radians(longitude),
                math.radians(longitude_std),
                method
            )
            self.__sinh.append(math.sin(hs))
            self.__cosh.append(math.cos(hs))
            self.__hsdt.append(math.degrees(hs))
            self.__azsdt.append(math.degrees(A))
        
    
    def calc_and_set_nh(self, prev_sinh, ndt, latitude, longitude):

        # nh データの作成
        self.__nh = calc_Nh(self.sinh, prev_sinh, ndt)
    
    def calc_and_set_sdhm(self, next_data, ndt):
        # 法線面直達日射量 Sddhm (kcal/m2)
        self.__sddhm = [calc_Sdhm(mm, ndt, self.sinh[mm],
                                  self.direct, next_data.direct,
                                  self.nh, next_data.nh) for mm in range(ndt)]

        # 水平面天空日射量 Ssdhm (kcal/m2)
        self.__ssdhm = [calc_Sdhm(mm, ndt, self.sinh[mm],
                                  self.sky, next_data.sky,
                                  self.nh, next_data.nh) for mm in range(ndt)]
        
    def get_as_list(self):
        return [self.__day_time, self.__direct, self.__sky, self.__hc_period]
    
    # 法線面直達日射量 Sddhm (kcal/m2)
    @property
    def sddhm(self):
        return self.__sddhm
    
    # 水平面天空日射量 Ssdhm (kcal/m2)
    @property
    def ssdhm(self):
        return self.__ssdhm

    # 太陽方位角 Azsdt (deg)
    @property
    def azsdt(self):
        return self.__azsdt

    # 太陽高度 hsdt
    @property
    def hsdt(self):
        return self.__hsdt
    
    # 太陽高度 hsdt (deg) の余弦
    @property
    def cosh(self):
        return self.__cosh
    
    # 太陽高度 hsdt (deg) の正弦
    @property
    def sinh(self):
        return self.__sinh
    
    # 時刻(h)
    @property
    def tt(self):
        return self.__tt
    
    # 太陽が出ている時間の(分割数に対する)数
    @property
    def nh(self):
        return self.__nh

    @property
    def index(self):
        return self.__index
    
    @property
    def nday(self):
        return self.__nday
    
    @property
    def nhour(self):
        return self.__nhour
    
    @property
    def day_time(self):
        return self.__day_time
    
    @property
    def direct(self):
        return self.__direct
    
    @property
    def sky(self):
        return self.__sky
    
    @property
    def hc_period(self):
        return self.__hc_period

## Climate

In [16]:
class Climate():
    
    def __init__(self, tag, ndt):
        
        self.__ndt = ndt

        # 地点データのロード
        zone, city, latitude, longitude, SRFileName, heating_start, heating_end, cooling_start, cooling_end = input_Point(tag)
        self.__zone = zone
        self.__city = city
        self.__latitude = latitude
        self.__longitude = longitude
        self.__heating_start = heating_start
        self.__heating_end = heating_end
        self.__cooling_start = cooling_start
        self.__cooling_end = cooling_end

        # 日射データのロード
        sr_data = input_SRData(SRFileName, heating_start, heating_end, cooling_start, cooling_end)
        sun_data = [SunData(s[0], s[1], s[2], s[3], s[4], latitude, longitude, ndt)
                    for s in sr_data] # 8761 個の行を持つ。
        self.__sun_data = sun_data
        
        for i in range(8760):
            if i == 0:
                prev_sinh = self.__sun_data[8759].sinh
            else:
                prev_sinh = self.__sun_data[i-1].sinh
            self.__sun_data[i].calc_and_set_nh(prev_sinh, ndt, latitude, longitude) 
        
        # 日射量の計算(日射量の計算は1~8760回行う。8761回目は行わないことに注意！)
        for i in range(8760):
            if i == 8759:
                next_data = self.__sun_data[0]
            else:
                next_data = self.__sun_data[i+1]
            self.__sun_data[i].calc_and_set_sdhm(next_data, ndt)

    # 法線面直達日射量 Sddhm (kcal/m2)
    def calc_sddhm(self, index, mm):
        return self.get_sun_data(index).sddhm[mm]

    # 水平面天空日射量 Ssdhm (kcal/m2)
    def calc_ssdhm(self, index, mm):
        return self.get_sun_data(index).ssdhm[mm]

    # 太陽方位角 Azsdt (deg)
    def calc_azsdt(self, index, mm):
        return self.get_sun_data(index).azsdt[mm]
            
    # 太陽高度 hsdt (deg) の正弦
    def calc_sinh(self, index, mm):
        return self.get_sun_data(index).sinh[mm]
    
    # 太陽高度 hsdt (deg) の余弦
    def calc_cosh(self, index, mm):
        return self.get_sun_data(index).cosh[mm]
    
    # 太陽高度 hsdt
    def calc_hsdt(self, index, mm):
        return self.get_sun_data(index).hsdt[mm]
    
    def get_month(self, index):
        return calc_Month(self.get_sun_data(index).day_time)
    
    def get_nh(self, index):
        return self.get_sun_data(index).nh
    
    def get_sun_data(self, index):
        return self.__sun_data[index]
    
    @property
    def zone(self):
        return self.__zone
    
    @property
    def city(self):
        return self.__city
    
    @property
    def latitude(self):
        return self.__latitude
    
    @property
    def longitude(self):
        return self.__longitude
    
    @property
    def heating_start(self):
        return self.__heating_start
    
    @property
    def heating_end(self):
        return self.__heating_end
    
    @property
    def cooling_start(self):
        return self.__cooling_start
    
    @property
    def cooling_end(self):
        return self.__cooling_end
    
    @property
    def sun_data(self):
        return self.__sun_data
    
    @property
    def ndt(self):
        return self.__ndt
    

#### Example

In [17]:
for i in range(1,9):
    ndt = 6
    c = Climate(i, ndt)
    print([c.zone, c.city, c.latitude, c.longitude, c.heating_start, c.heating_end, c.cooling_start, c.cooling_end ])

[1, '北見', 43.82, 143.91, 110100, 43024, 50100, 103124]
[2, '岩見沢', 43.21, 141.78833, 110100, 43024, 50100, 103124]
[3, '盛岡', 39.695, 141.168333333333, 120100, 33124, 40100, 113024]
[4, '長野', 36.66, 138.195, 120100, 33124, 40100, 113024]
[5, '宇都宮', 36.5466666666667, 139.871666666667, 120100, 33124, 40100, 113024]
[6, '岡山', 34.6583333333333, 133.918333333333, 120100, 33124, 40100, 113024]
[7, '宮崎', 31.935, 131.416666666667, 120100, 33124, 40100, 113024]
[8, '那覇', 26.2033333333333, 127.688333333333, 10100, 33124, 40100, 123124]


In [18]:
for i in range(1,9):
    
    ndt = 6
    c = Climate(i, ndt)
    
    HStart, HEnd, CStart, CEnd = c.heating_start, c.heating_end, c.cooling_start, c.cooling_end
    
    print('region: ' + str(i))
    
    print('Expected start of heating period: ' + str(HStart) )
    print('Expected end of heating period: ' + str(HEnd) )
    print('Expected start of cooling period: ' + str(CStart) )
    print('Expected end of cooling period: ' + str(CEnd) )
    
    for i, d in enumerate(c.sun_data):
        d_mns = c.sun_data[i-1]
        if d.hc_period != d_mns.hc_period:
            print(d_mns.day_time, d_mns.hc_period, d.day_time, d.hc_period)    

region: 1
Expected start of heating period: 110100
Expected end of heating period: 43024
Expected start of cooling period: 50100
Expected end of cooling period: 103124
43024 1 50101 2
103124 2 110101 1
region: 2
Expected start of heating period: 110100
Expected end of heating period: 43024
Expected start of cooling period: 50100
Expected end of cooling period: 103124
43024 1 50101 2
103124 2 110101 1
region: 3
Expected start of heating period: 120100
Expected end of heating period: 33124
Expected start of cooling period: 40100
Expected end of cooling period: 113024
33124 1 40101 2
113024 2 120101 1
region: 4
Expected start of heating period: 120100
Expected end of heating period: 33124
Expected start of cooling period: 40100
Expected end of cooling period: 113024
33124 1 40101 2
113024 2 120101 1
region: 5
Expected start of heating period: 120100
Expected end of heating period: 33124
Expected start of cooling period: 40100
Expected end of cooling period: 113024
33124 1 40101 2
113024 2